In [1]:
import torch
import torch.nn.functional
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv("labeled_file.txt", sep="\t", header=None)
test_data = pd.read_csv("unlabeled_file.txt", sep="\t", header=None)
test_data_bk = test_data

In [3]:
train_data = train_data.sample(frac=1).reset_index(drop=True)

In [4]:
layers_dim = [21, 128, 128, 128]

In [5]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = torch.nn.Linear(layers_dim[0], layers_dim[1])
        self.relu1 = torch.nn.ReLU()
        self.drop = torch.nn.Dropout(0.4)
        self.fc2 = torch.nn.Linear(layers_dim[1], layers_dim[2])
        self.fc3 = torch.nn.Linear(layers_dim[2], layers_dim[3])
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.relu1(x)
        x = self.drop(x)
        x = self.fc3(x)
        
        return x
    
net = Net()

In [6]:
#USE THIS ONLY FOR LOADING THE MODEL

netLOADER = torch.nn.Sequential(
    torch.nn.Linear(layers_dim[0], layers_dim[1]),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(layers_dim[1], layers_dim[2]),
)

In [7]:
train_x = train_data.iloc[:, 2:-1]
test_x = test_data.iloc[:, 2:-1]
train_y = train_data.iloc[:, -1]

train_x = np.array(train_x.values, dtype=np.float32)
train_y = np.array(train_y.values, dtype=np.long)
test_x = np.array(test_x.values, dtype=np.float32)

In [8]:
split_ratio = 0.8   #train_size/(train_size+val_size)
assert(split_ratio >= 0.0 and split_ratio <= 1.0)
index = (int)(split_ratio * train_x.shape[0])

In [9]:
train_val_x = train_x[index:, :]
train_val_y = train_y[index:]
train_x = train_x[:index, :]
train_y = train_y[:index]
print(train_x.shape, train_y.shape, train_val_x.shape, train_val_y.shape)

(3708, 21) (3708,) (928, 21) (928,)


In [10]:
print(net.parameters)

<bound method Module.parameters of Net(
  (fc1): Linear(in_features=21, out_features=128, bias=True)
  (relu1): ReLU()
  (drop): Dropout(p=0.4)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
)>


In [11]:
train_x = torch.from_numpy(train_x)
test_x = torch.from_numpy(test_x)
train_y = torch.from_numpy(train_y)
train_val_x = torch.from_numpy(train_val_x)
train_val_y = torch.from_numpy(train_val_y)

In [12]:
print(train_y.shape)
print(train_val_y.shape)

torch.Size([3708])
torch.Size([928])


In [13]:
train_y = torch.tensor(train_y, dtype=torch.long)
train_val_y = torch.tensor(train_val_y, dtype=torch.long)

C:\Users\Saraansh Chopra\AppData\Local\conda\conda\envs\torch-gpu\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
C:\Users\Saraansh Chopra\AppData\Local\conda\conda\envs\torch-gpu\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [14]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
num_epochs = 5000

In [15]:
for epoch in range(num_epochs):  # number of epochs
    outputs = net(train_x)
    loss = criterion(outputs, train_y)
    
    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/5000], Loss: 133.3223
Epoch [2/5000], Loss: 75.7861
Epoch [3/5000], Loss: 56.9665
Epoch [4/5000], Loss: 41.5990
Epoch [5/5000], Loss: 28.7331
Epoch [6/5000], Loss: 17.9247
Epoch [7/5000], Loss: 12.3046
Epoch [8/5000], Loss: 8.5382
Epoch [9/5000], Loss: 5.9507
Epoch [10/5000], Loss: 5.3762
Epoch [11/5000], Loss: 4.8733
Epoch [12/5000], Loss: 4.6441
Epoch [13/5000], Loss: 4.7675
Epoch [14/5000], Loss: 4.5131
Epoch [15/5000], Loss: 4.4777
Epoch [16/5000], Loss: 4.4918
Epoch [17/5000], Loss: 4.4506
Epoch [18/5000], Loss: 4.4032
Epoch [19/5000], Loss: 4.4010
Epoch [20/5000], Loss: 4.3471
Epoch [21/5000], Loss: 4.3199
Epoch [22/5000], Loss: 4.2795
Epoch [23/5000], Loss: 4.2190
Epoch [24/5000], Loss: 4.2103
Epoch [25/5000], Loss: 4.2066
Epoch [26/5000], Loss: 4.1366
Epoch [27/5000], Loss: 4.1048
Epoch [28/5000], Loss: 4.0903
Epoch [29/5000], Loss: 4.0392
Epoch [30/5000], Loss: 4.0363
Epoch [31/5000], Loss: 4.1542
Epoch [32/5000], Loss: 4.0564
Epoch [33/5000], Loss: 3.9602
Epoch [34/5

Epoch [275/5000], Loss: 2.9133
Epoch [276/5000], Loss: 2.9223
Epoch [277/5000], Loss: 2.9210
Epoch [278/5000], Loss: 2.9171
Epoch [279/5000], Loss: 2.9312
Epoch [280/5000], Loss: 2.9357
Epoch [281/5000], Loss: 2.9142
Epoch [282/5000], Loss: 2.9089
Epoch [283/5000], Loss: 2.8847
Epoch [284/5000], Loss: 2.9126
Epoch [285/5000], Loss: 2.8728
Epoch [286/5000], Loss: 2.8719
Epoch [287/5000], Loss: 2.8768
Epoch [288/5000], Loss: 2.8834
Epoch [289/5000], Loss: 2.8491
Epoch [290/5000], Loss: 2.8535
Epoch [291/5000], Loss: 2.8805
Epoch [292/5000], Loss: 2.8476
Epoch [293/5000], Loss: 2.8453
Epoch [294/5000], Loss: 2.8485
Epoch [295/5000], Loss: 2.8340
Epoch [296/5000], Loss: 2.8271
Epoch [297/5000], Loss: 2.8149
Epoch [298/5000], Loss: 2.8431
Epoch [299/5000], Loss: 2.8251
Epoch [300/5000], Loss: 2.8541
Epoch [301/5000], Loss: 2.8473
Epoch [302/5000], Loss: 2.8656
Epoch [303/5000], Loss: 2.8470
Epoch [304/5000], Loss: 2.8420
Epoch [305/5000], Loss: 2.8182
Epoch [306/5000], Loss: 2.8430
Epoch [3

Epoch [543/5000], Loss: 2.8216
Epoch [544/5000], Loss: 2.8182
Epoch [545/5000], Loss: 2.8035
Epoch [546/5000], Loss: 2.8086
Epoch [547/5000], Loss: 2.8059
Epoch [548/5000], Loss: 2.8112
Epoch [549/5000], Loss: 2.8160
Epoch [550/5000], Loss: 2.7930
Epoch [551/5000], Loss: 2.7980
Epoch [552/5000], Loss: 2.7933
Epoch [553/5000], Loss: 2.7818
Epoch [554/5000], Loss: 2.8112
Epoch [555/5000], Loss: 2.7972
Epoch [556/5000], Loss: 2.8000
Epoch [557/5000], Loss: 2.8065
Epoch [558/5000], Loss: 2.7881
Epoch [559/5000], Loss: 2.7775
Epoch [560/5000], Loss: 2.7724
Epoch [561/5000], Loss: 2.7812
Epoch [562/5000], Loss: 2.7777
Epoch [563/5000], Loss: 2.7719
Epoch [564/5000], Loss: 2.7652
Epoch [565/5000], Loss: 2.7719
Epoch [566/5000], Loss: 2.7470
Epoch [567/5000], Loss: 2.7646
Epoch [568/5000], Loss: 2.7519
Epoch [569/5000], Loss: 2.7035
Epoch [570/5000], Loss: 2.7380
Epoch [571/5000], Loss: 2.7316
Epoch [572/5000], Loss: 2.7126
Epoch [573/5000], Loss: 2.7070
Epoch [574/5000], Loss: 2.7121
Epoch [5

Epoch [811/5000], Loss: 2.8418
Epoch [812/5000], Loss: 2.8723
Epoch [813/5000], Loss: 2.8500
Epoch [814/5000], Loss: 2.8629
Epoch [815/5000], Loss: 2.8919
Epoch [816/5000], Loss: 2.8729
Epoch [817/5000], Loss: 2.8716
Epoch [818/5000], Loss: 2.8573
Epoch [819/5000], Loss: 2.8619
Epoch [820/5000], Loss: 2.8475
Epoch [821/5000], Loss: 2.8689
Epoch [822/5000], Loss: 2.8495
Epoch [823/5000], Loss: 2.8352
Epoch [824/5000], Loss: 2.8285
Epoch [825/5000], Loss: 2.8158
Epoch [826/5000], Loss: 2.7930
Epoch [827/5000], Loss: 2.7873
Epoch [828/5000], Loss: 2.7949
Epoch [829/5000], Loss: 2.7283
Epoch [830/5000], Loss: 2.7362
Epoch [831/5000], Loss: 2.7199
Epoch [832/5000], Loss: 2.7133
Epoch [833/5000], Loss: 2.6947
Epoch [834/5000], Loss: 2.7205
Epoch [835/5000], Loss: 2.7077
Epoch [836/5000], Loss: 2.7070
Epoch [837/5000], Loss: 2.6980
Epoch [838/5000], Loss: 2.6963
Epoch [839/5000], Loss: 2.6887
Epoch [840/5000], Loss: 2.6993
Epoch [841/5000], Loss: 2.7093
Epoch [842/5000], Loss: 2.6681
Epoch [8

Epoch [1079/5000], Loss: 2.5546
Epoch [1080/5000], Loss: 2.5526
Epoch [1081/5000], Loss: 2.5527
Epoch [1082/5000], Loss: 2.5414
Epoch [1083/5000], Loss: 2.5477
Epoch [1084/5000], Loss: 2.5438
Epoch [1085/5000], Loss: 2.5441
Epoch [1086/5000], Loss: 2.5589
Epoch [1087/5000], Loss: 2.5448
Epoch [1088/5000], Loss: 2.5499
Epoch [1089/5000], Loss: 2.5546
Epoch [1090/5000], Loss: 2.5305
Epoch [1091/5000], Loss: 2.5327
Epoch [1092/5000], Loss: 2.5306
Epoch [1093/5000], Loss: 2.5423
Epoch [1094/5000], Loss: 2.5392
Epoch [1095/5000], Loss: 2.5443
Epoch [1096/5000], Loss: 2.5373
Epoch [1097/5000], Loss: 2.5326
Epoch [1098/5000], Loss: 2.5275
Epoch [1099/5000], Loss: 2.5479
Epoch [1100/5000], Loss: 2.5387
Epoch [1101/5000], Loss: 2.5296
Epoch [1102/5000], Loss: 2.5568
Epoch [1103/5000], Loss: 2.5453
Epoch [1104/5000], Loss: 2.5439
Epoch [1105/5000], Loss: 2.5426
Epoch [1106/5000], Loss: 2.5602
Epoch [1107/5000], Loss: 2.5641
Epoch [1108/5000], Loss: 2.5534
Epoch [1109/5000], Loss: 2.5294
Epoch [1

...

Epoch [3924/5000], Loss: 2.5039
Epoch [3925/5000], Loss: 2.5049
Epoch [3926/5000], Loss: 2.5175
Epoch [3927/5000], Loss: 2.4961
Epoch [3928/5000], Loss: 2.5039
Epoch [3929/5000], Loss: 2.4998
Epoch [3930/5000], Loss: 2.5099
Epoch [3931/5000], Loss: 2.4956
Epoch [3932/5000], Loss: 2.4800
Epoch [3933/5000], Loss: 2.4930
Epoch [3934/5000], Loss: 2.5050
Epoch [3935/5000], Loss: 2.4973
Epoch [3936/5000], Loss: 2.5045
Epoch [3937/5000], Loss: 2.4917
Epoch [3938/5000], Loss: 2.4801
Epoch [3939/5000], Loss: 2.4912
Epoch [3940/5000], Loss: 2.5142
Epoch [3941/5000], Loss: 2.5023
Epoch [3942/5000], Loss: 2.5259
Epoch [3943/5000], Loss: 2.5009
Epoch [3944/5000], Loss: 2.5024
Epoch [3945/5000], Loss: 2.4933
Epoch [3946/5000], Loss: 2.4974
Epoch [3947/5000], Loss: 2.4831
Epoch [3948/5000], Loss: 2.5180
Epoch [3949/5000], Loss: 2.4823
Epoch [3950/5000], Loss: 2.5047
Epoch [3951/5000], Loss: 2.5174
Epoch [3952/5000], Loss: 2.4974
Epoch [3953/5000], Loss: 2.5057
Epoch [3954/5000], Loss: 2.4817
Epoch [3

Epoch [4184/5000], Loss: 2.6130
Epoch [4185/5000], Loss: 2.6027
Epoch [4186/5000], Loss: 2.6145
Epoch [4187/5000], Loss: 2.5787
Epoch [4188/5000], Loss: 2.5690
Epoch [4189/5000], Loss: 2.5972
Epoch [4190/5000], Loss: 2.6240
Epoch [4191/5000], Loss: 2.5905
Epoch [4192/5000], Loss: 2.6253
Epoch [4193/5000], Loss: 2.6083
Epoch [4194/5000], Loss: 2.6231
Epoch [4195/5000], Loss: 2.6167
Epoch [4196/5000], Loss: 2.5731
Epoch [4197/5000], Loss: 2.6660
Epoch [4198/5000], Loss: 2.5951
Epoch [4199/5000], Loss: 2.6543
Epoch [4200/5000], Loss: 2.6695
Epoch [4201/5000], Loss: 2.6707
Epoch [4202/5000], Loss: 2.6371
Epoch [4203/5000], Loss: 2.6293
Epoch [4204/5000], Loss: 2.5897
Epoch [4205/5000], Loss: 2.6696
Epoch [4206/5000], Loss: 2.5867
Epoch [4207/5000], Loss: 2.6531
Epoch [4208/5000], Loss: 2.6380
Epoch [4209/5000], Loss: 2.6529
Epoch [4210/5000], Loss: 2.6252
Epoch [4211/5000], Loss: 2.5636
Epoch [4212/5000], Loss: 2.6598
Epoch [4213/5000], Loss: 2.5682
Epoch [4214/5000], Loss: 2.6212
Epoch [4

Epoch [4442/5000], Loss: 2.3826
Epoch [4443/5000], Loss: 2.3787
Epoch [4444/5000], Loss: 2.4013
Epoch [4445/5000], Loss: 2.3652
Epoch [4446/5000], Loss: 2.3928
Epoch [4447/5000], Loss: 2.4343
Epoch [4448/5000], Loss: 2.4428
Epoch [4449/5000], Loss: 2.4980
Epoch [4450/5000], Loss: 2.5336
Epoch [4451/5000], Loss: 2.5251
Epoch [4452/5000], Loss: 2.4065
Epoch [4453/5000], Loss: 2.9046
Epoch [4454/5000], Loss: 2.4924
Epoch [4455/5000], Loss: 2.5219
Epoch [4456/5000], Loss: 2.5183
Epoch [4457/5000], Loss: 2.5075
Epoch [4458/5000], Loss: 2.5007
Epoch [4459/5000], Loss: 2.5121
Epoch [4460/5000], Loss: 2.4966
Epoch [4461/5000], Loss: 2.5090
Epoch [4462/5000], Loss: 2.5231
Epoch [4463/5000], Loss: 2.5054
Epoch [4464/5000], Loss: 2.5049
Epoch [4465/5000], Loss: 2.4967
Epoch [4466/5000], Loss: 2.4947
Epoch [4467/5000], Loss: 2.4815
Epoch [4468/5000], Loss: 2.4948
Epoch [4469/5000], Loss: 2.4749
Epoch [4470/5000], Loss: 2.5085
Epoch [4471/5000], Loss: 2.5219
Epoch [4472/5000], Loss: 2.5349
Epoch [4

Epoch [4704/5000], Loss: 2.7028
Epoch [4705/5000], Loss: 2.7299
Epoch [4706/5000], Loss: 2.6943
Epoch [4707/5000], Loss: 2.7085
Epoch [4708/5000], Loss: 2.7074
Epoch [4709/5000], Loss: 2.7283
Epoch [4710/5000], Loss: 2.7362
Epoch [4711/5000], Loss: 2.7444
Epoch [4712/5000], Loss: 2.7594
Epoch [4713/5000], Loss: 2.7578
Epoch [4714/5000], Loss: 2.7614
Epoch [4715/5000], Loss: 2.7899
Epoch [4716/5000], Loss: 2.7779
Epoch [4717/5000], Loss: 2.7845
Epoch [4718/5000], Loss: 2.7842
Epoch [4719/5000], Loss: 2.7707
Epoch [4720/5000], Loss: 2.7762
Epoch [4721/5000], Loss: 2.7740
Epoch [4722/5000], Loss: 2.7579
Epoch [4723/5000], Loss: 2.7391
Epoch [4724/5000], Loss: 2.8022
Epoch [4725/5000], Loss: 2.7152
Epoch [4726/5000], Loss: 2.7661
Epoch [4727/5000], Loss: 2.7673
Epoch [4728/5000], Loss: 2.7297
Epoch [4729/5000], Loss: 2.7365
Epoch [4730/5000], Loss: 2.7269
Epoch [4731/5000], Loss: 2.7747
Epoch [4732/5000], Loss: 2.7368
Epoch [4733/5000], Loss: 2.7302
Epoch [4734/5000], Loss: 2.7585
Epoch [4

Epoch [4961/5000], Loss: 2.5567
Epoch [4962/5000], Loss: 2.5560
Epoch [4963/5000], Loss: 2.5383
Epoch [4964/5000], Loss: 2.5319
Epoch [4965/5000], Loss: 2.5237
Epoch [4966/5000], Loss: 2.5431
Epoch [4967/5000], Loss: 2.5406
Epoch [4968/5000], Loss: 2.5477
Epoch [4969/5000], Loss: 2.5564
Epoch [4970/5000], Loss: 2.5469
Epoch [4971/5000], Loss: 2.5302
Epoch [4972/5000], Loss: 2.5376
Epoch [4973/5000], Loss: 2.5492
Epoch [4974/5000], Loss: 2.5274
Epoch [4975/5000], Loss: 2.5393
Epoch [4976/5000], Loss: 2.5480
Epoch [4977/5000], Loss: 2.5427
Epoch [4978/5000], Loss: 2.5362
Epoch [4979/5000], Loss: 2.5402
Epoch [4980/5000], Loss: 2.5342
Epoch [4981/5000], Loss: 2.5437
Epoch [4982/5000], Loss: 2.5403
Epoch [4983/5000], Loss: 2.5392
Epoch [4984/5000], Loss: 2.5563
Epoch [4985/5000], Loss: 2.5284
Epoch [4986/5000], Loss: 2.5353
Epoch [4987/5000], Loss: 2.5497
Epoch [4988/5000], Loss: 2.5356
Epoch [4989/5000], Loss: 2.5316
Epoch [4990/5000], Loss: 2.5399
Epoch [4991/5000], Loss: 2.5333
Epoch [4

In [16]:
def predict_accuracy(val_x, val_y):
    outputs = net(val_x)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == val_y).sum().item()
    print("Accuracy = {}%".format(100 *(float(correct/val_x.shape[0]))))

In [17]:
#outputs = net(train_x)

In [18]:
#_, predicted = torch.max(outputs, 1)

In [19]:
#print(predicted)

In [20]:
#correct = (predicted == train_y).sum().item()

In [21]:
#print("Accuracy = {}%".format(100 *(float(correct/train_data.shape[0]))))

In [22]:
predict_accuracy(train_x, train_y)
predict_accuracy(train_val_x, train_val_y)

Accuracy = 40.83063646170442%
Accuracy = 40.94827586206897%


In [ ]:
def generate_labels(test_x):
    outputs = net(test_x)
    _, predictions = torch.max(outputs, 1)
    return predictions

In [ ]:
torch.save(net, "trained_model_10000_epochs.pt")

In [ ]:
netLOADER = torch.load("trained_model_10000_epochs.pt")

In [ ]:
test_labels = generate_labels(test_x)
print(test_labels)

In [ ]:
test_data[[23]] = test_labels
test_data

In [ ]:
test_data.to_csv("Labelled_prev_unlabelBCLL.txt", sep=" ", header=False, index=False)